## Import

In [14]:
from sklearn import preprocessing
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import * 
from keras.preprocessing import image
from keras.models import Sequential
# To initialize neural network
from keras.layers import Convolution2D
# Images are two dimensional, concolution step
from keras.layers import MaxPooling2D
# Pooling step
from keras.layers import Flatten
# Convert pools feature map into this large feature vector
from keras.layers import Dense
#To add fully connected layers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import decode_predictions
from keras.applications.xception import Xception
#from keras.applications.InceptionResNetV2 import InceptionResNetV2
from keras import applications
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
# To initialize neural network
from keras.layers import Convolution2D
# Images are two dimensional, concolution step
from keras.layers import MaxPooling2D
# Pooling step
from keras.layers import Flatten
# Convert pools feature map into this large feature vector
from keras.layers import Dense
#To add fully connected layers
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications.resnet50 import ResNet50
from keras.applications import resnet50
from keras.models import Model
import keras
from keras.optimizers import Adam
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout

from keras.callbacks import LearningRateScheduler

import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import GridSearchCV

## Data Preparation

In [21]:
SIZE = 224
TRAIN_DIR= 'imagesdata/train/'
train_images = []
train_labels = [] 
for img_label in (os.listdir(TRAIN_DIR)):
    img_dir=os.path.join(TRAIN_DIR,img_label)
    for img_name in (os.listdir(img_dir)):
        img_path=os.path.join(img_dir,img_name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(img_label)
train_images = np.array(train_images)
train_labels = np.array(train_labels)
SIZE = 224
TEST_DIR= 'imagesdata/test/'
test_images = []
test_labels = [] 
for img_label in (os.listdir(TEST_DIR)):
    img_dir=os.path.join(TEST_DIR,img_label)
    for img_name in (os.listdir(img_dir)):
        img_path=os.path.join(img_dir,img_name)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(img_label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

#########################################################
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#Split data into test and train datasets (already split but assigning to meaningful convention)

x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

###################################################################
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encode y values for neural network. 
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [22]:
print(y_test_one_hot[150:200])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [34]:
#IMAGE_SIZE = 224
pretrained_model =applications.Xception(input_shape=(224,224, 3), include_top=False)
pretrained_model.trainable = False

model = Sequential([
    pretrained_model,
    Flatten(),
    Dense(2, activation='softmax')
])

In [35]:
def define_model(learning_rate=0.01, momentum=0.1, activation='relu', init_weights='uniform', optimizer='Adam'):   
    pretrained_model =applications.Xception(input_shape=(224,224, 3), include_top=False)
    pretrained_model.trainable = False

    model = Sequential([
    pretrained_model,
    Flatten(),
    Dense(2, activation='softmax')
    ])

    
    # compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,      
                  metrics=['accuracy'])
    return model

In [ ]:
#Take a subset of train for grid search. Let us take 10% for now
from sklearn.model_selection import train_test_split
x_grid, x_not_use, y_grid, y_not_use = train_test_split(x_train, y_train, test_size=0.9, random_state=42)


from keras.wrappers.scikit_learn import KerasClassifier
batch_size = 32
epochs = 10

model = KerasClassifier(build_fn=define_model, 
                        epochs=epochs, 
                        batch_size = batch_size, 
                        verbose=1)

learning_rate = [0.0001, 0.001, 0.01, 0.1]
momentum = [0.3, 0.5, 0.7, 0.9]

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3

init_weights = ['uniform', 'normal', 'he_uniform']
#Also try lecun_uniform, he_normal, glorot_normal, etc. 
optimizer = ['SGD', 'RMSprop', 'Adam']

param_grid = dict( init_weights=init_weights, optimizer=optimizer,learning_rate=learning_rate, momentum=momentum)

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=None, cv=3)


grid_result = grid.fit(x_grid, y_grid)

# summarize results
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/10
123/123 [==============================] - 36s 294ms/step - loss: 4.4247 - accuracy: 0.5366
Epoch 2/10
123/123 [==============================] - 33s 267ms/step - loss: 3.7516 - accuracy: 0.6341
Epoch 3/10
123/123 [==============================] - 33s 268ms/step - loss: 2.4327 - accuracy: 0.6504
Epoch 4/10
123/123 [==============================] - 33s 269ms/step - loss: 3.1497 - accuracy: 0.6992
Epoch 5/10
123/123 [==============================] - 33s 267ms/step - loss: 0.4205 - accuracy: 0.9187
Epoch 6/10
123/123 [==============================] - 33s 267ms/step - loss: 0.0848 - accuracy: 0.9675
Epoch 7/10
123/123 [==============================] - 33s 269ms/step - loss: 0.0102 - accuracy: 1.0000
Epoch 8/10
123/123 [==============================] - 33s 267ms/step - loss: 0.0113 - accuracy: 1.0000
Epoch 9/10
123/123 [==============================] - 33s 265ms/step - loss: 0.0199 - accuracy: 0.9919
Epoch 10/10
62/62 [==============================] - 18s 293ms/step
Epoch

123/123 [==============================] - 43s 348ms/step - loss: 0.0530 - accuracy: 0.9919
Epoch 7/10
123/123 [==============================] - 43s 351ms/step - loss: 0.0063 - accuracy: 1.0000
Epoch 8/10
123/123 [==============================] - 43s 346ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 9/10
123/123 [==============================] - 38s 309ms/step - loss: 8.7656e-04 - accuracy: 1.0000
Epoch 10/10
62/62 [==============================] - 21s 343ms/step
Epoch 1/10
124/124 [==============================] - 41s 329ms/step - loss: 3.4052 - accuracy: 0.6290
Epoch 2/10
124/124 [==============================] - 38s 307ms/step - loss: 2.7369 - accuracy: 0.6048
Epoch 3/10
124/124 [==============================] - 38s 308ms/step - loss: 1.3773 - accuracy: 0.7419
Epoch 4/10
124/124 [==============================] - 38s 309ms/step - loss: 0.1953 - accuracy: 0.9032
Epoch 5/10
124/124 [==============================] - 39s 316ms/step - loss: 0.6605 - accuracy: 0.8468
Epoch 6/10
1

123/123 [==============================] - 42s 342ms/step - loss: 2.5199 - accuracy: 0.5203
Epoch 2/10
123/123 [==============================] - 38s 312ms/step - loss: 1.0509 - accuracy: 0.7805
Epoch 3/10
123/123 [==============================] - 38s 313ms/step - loss: 0.2490 - accuracy: 0.9187
Epoch 4/10
123/123 [==============================] - 38s 311ms/step - loss: 0.4343 - accuracy: 0.9024
Epoch 5/10
123/123 [==============================] - 38s 310ms/step - loss: 0.1670 - accuracy: 0.9593
Epoch 6/10
123/123 [==============================] - 39s 317ms/step - loss: 0.1101 - accuracy: 0.9593
Epoch 7/10
123/123 [==============================] - 38s 306ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 8/10
123/123 [==============================] - 39s 315ms/step - loss: 0.0339 - accuracy: 0.9756
Epoch 9/10
123/123 [==============================] - 39s 318ms/step - loss: 0.0054 - accuracy: 1.0000
Epoch 10/10
62/62 [==============================] - 22s 348ms/step
Epoch 1/10
123/1

123/123 [==============================] - 33s 265ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 7/10
123/123 [==============================] - 33s 267ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 8/10
123/123 [==============================] - 33s 266ms/step - loss: 0.0144 - accuracy: 0.9919
Epoch 9/10
123/123 [==============================] - 33s 267ms/step - loss: 0.0077 - accuracy: 1.0000
Epoch 10/10
62/62 [==============================] - 18s 293ms/step
Epoch 1/10
124/124 [==============================] - 36s 291ms/step - loss: 9.0277 - accuracy: 0.5484
Epoch 2/10
124/124 [==============================] - 33s 267ms/step - loss: 2.6362 - accuracy: 0.7581
Epoch 3/10
124/124 [==============================] - 33s 266ms/step - loss: 0.5246 - accuracy: 0.9032
Epoch 4/10
124/124 [==============================] - 33s 267ms/step - loss: 0.0483 - accuracy: 0.9919
Epoch 5/10
124/124 [==============================] - 33s 265ms/step - loss: 0.0253 - accuracy: 0.9919
Epoch 6/10
124/1

123/123 [==============================] - 37s 298ms/step - loss: 10.7957 - accuracy: 0.5122
Epoch 2/10
123/123 [==============================] - 33s 267ms/step - loss: 2.3445 - accuracy: 0.7317
Epoch 3/10
123/123 [==============================] - 34s 273ms/step - loss: 0.9046 - accuracy: 0.8780
Epoch 4/10
123/123 [==============================] - 33s 268ms/step - loss: 0.0552 - accuracy: 0.9756
Epoch 5/10
123/123 [==============================] - 33s 267ms/step - loss: 0.0371 - accuracy: 0.9675
Epoch 6/10
123/123 [==============================] - 33s 268ms/step - loss: 0.0920 - accuracy: 0.9756
Epoch 7/10
123/123 [==============================] - 33s 267ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 8/10
123/123 [==============================] - 33s 267ms/step - loss: 0.0149 - accuracy: 0.9919
Epoch 9/10
123/123 [==============================] - 33s 270ms/step - loss: 0.0104 - accuracy: 0.9919
Epoch 10/10
62/62 [==============================] - 19s 299ms/step
Epoch 1/10
123/

In [31]:
from keras.wrappers.scikit_learn import KerasClassifier
batch_size = 32
epochs = 10

model = KerasClassifier(build_fn=define_model, 
                        epochs=epochs, 
                        batch_size = batch_size, 
                        verbose=1)

learning_rate = [0.0001, 0.001, 0.01, 0.1]
momentum = [0.3, 0.5, 0.7, 0.9]

#n_jobs=16 uses 16 CPUs. Try not to do -1 on your system as it may hang!!!
# -1 refers to using all available CPUs
#Cross validation, cv=3
param_grid = dict(learning_rate=learning_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=16, cv=3)

grid_result = grid.fit(x_grid, y_grid)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

ValueError: learning_rate is not a legal parameter

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean = %f (std=%f) with: %r" % (mean, stdev, param))